In [2]:

"""
Módulo de funções de ativação para redes neurais.

Cada função de ativação tem:
- forward: calcula a ativação
- derivada: usada no backpropagation (veremos depois)
"""

import numpy as np

class Ativacao:
    """"Classe base para funções de ativação. """

    def forward(self, x):
        """Aplica a função de ativação """
        raise NotImplementedError

    def derivada(self, x):
        """Calcula a derivada (para backpropagation) """
        raise NotImplementedError


In [36]:
# === Funções de ativação

class Degrau(Ativacao):
    """
    Função Degrau (Step Function).
    Retorna 1 se x >= limiar, senão 0.
    """

    def __init__(self, limiar=1.0):
        self.limiar = limiar

    def forward(self, x):
        return 1 if x >= self.limiar else 0

    def derivada(self, x):
        # Degrau não é diferenciável, mas podemos retornar 0
        return 0

class Sigmoid(Ativacao):
    """
    Função Sigmoid (Logística).
    Mapeia valores para range (0, 1).
    """

    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def derivada(self, x):
        s = self.forward(x)
        return s * (1 - s)

class ReLU(Ativacao):
    """
    Rectified Linear Unit.
    Retorna max(0, x).
    """

    def forward(self, x):
        return max(0, x)

    def derivada(self, x):
        return 1 if 0 >= 0 else 0

class Tanh(Ativacao):
    """
    Tangente Hiperbólica.
    Mapeia valores para range (-1, 1).
    """

    def forward(self, x):
        return np.tanh(x)

    def derivada(self, x):
        return 1 - np.tanh(x)**2

class Linear(Ativacao):

    def forward(self, x):
        return x

    def derivada(self, x):
        s = self.foward(x)
        return s
    
    

In [37]:
# ==== Funções auxiliares para compatibilidade ====

def criar_ativacao(nome):
    """
    Factory function para criar ativações por nome.
    
    Args:
        nome (str): Nome da ativação ('degrau', 'sigmoid', 'relu', 'tanh')
    
    Returns:
        Ativacao: Objeto da função de ativação
    """

    ativacoes = {
        'degrau': Degrau,
        'sigmoid':Sigmoid,
        'relu': ReLU,
        'tanh': Tanh,
        'linear': Linear
    }

    if nome.lower() not in ativacoes:
        raise ValueError(f"Ativação '{nome}' não encontrada. Opções {list(ativacoes.keys())}")

    return ativacoes[nome.lower()]()

In [11]:
"""
Implementação de um neurônio artificial
"""

# from activations import Ativacao, Degrau

class Neuronio:
    """
    Um neurônio artificial com aprendizado supervisionado.
    
    Attributes:
        pesos (np.ndarray): Pesos sinápticos
        bias (float): Termo de viés
        taxa_aprendizado (float): Taxa de aprendizado
        ativacao (Ativacao): Função de ativação
    """
    
    def __init__(self, num_entradas, taxa_aprendizado=0.1, ativacao=None):
    
        """
        Inicializa o neurônio.
        
        Args:
            num_entradas (int): Número de entradas
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Degrau)
        """
        self.pesos = np.random.rand(num_entradas)
        self.bias = 0.0
        self.taxa_aprendizado = taxa_aprendizado
        
        # Se não passar ativação, usa Degrau por padrão
        self.ativacao = ativacao if ativacao is not None else Degrau()


    def forward(self, entrada):
        """
        Propagação para frente.
        
        Args:
            entrada: Vetor de entrada
            
        Returns:
            tuple: (saida_bruta, saida_ativada)
        """

        # Combinação linear
        saida_bruta = np.dot(entrada, self.pesos) + self.bias

        # Aplica ativação (agora vem do objeto de ativação)
        saida_ativada = self.ativacao.forward(saida_bruta)

        return saida_bruta, saida_ativada

    def treinar(self, entrada, esperado):
        """
        Treina o neurônio com um exemplo.
        
        Args:
            entrada: Vetor de entrada
            esperado: Saída esperada
            
        Returns:
            tuple: (saida, erro)
        """
        saida_bruta, saida = self.forward(entrada)
        erro = esperado - saida

        # Atualiza pesos
        self.pesos += self.taxa_aprendizado * erro * np.array(entrada)
        self.bias += self.taxa_aprendizado * erro

        return saida, erro

    def __repr__(self):
        """Representação em string do neurônio."""
        return f"Neuronio(entradas={len(self.pesos)}, ativacao={self.ativacao.__class__.__name__})"

        
    

In [40]:
"""
Script de teste para o neurônio com diferentes ativações.
"""

# === TEST 1: Neurônio com Degrau (padrão) ===
print("=" * 50)
print("TEST 1: Neurônio com função Degrau")
print("=" * 50)

data = [0,1,0,0,0,0,0,0,0,0]
esperado = 1

neuronio_degrau = Neuronio(num_entradas=10, taxa_aprendizado=0.1)
print(neuronio_degrau)

for epoca in range(10):
    saida, erro = neuronio_degrau.treinar(data, esperado)
    print(f"Época {epoca} | saída={saida} | erro={erro}")

TEST 1: Neurônio com função Degrau
Neuronio(entradas=10, ativacao=Degrau)
Época 0 | saída=0 | erro=1
Época 1 | saída=0 | erro=1
Época 2 | saída=0 | erro=1
Época 3 | saída=0 | erro=1
Época 4 | saída=0 | erro=1
Época 5 | saída=1 | erro=0
Época 6 | saída=1 | erro=0
Época 7 | saída=1 | erro=0
Época 8 | saída=1 | erro=0
Época 9 | saída=1 | erro=0


In [28]:
# ===== TESTE 2: Neurônio com Sigmoid =====
print("\n" + "=" * 50)
print("TESTE 2: Neurônio com função Sigmoid")
print("=" * 50)

neuronio_sigmoid = Neuronio(
    num_entradas=10, 
    taxa_aprendizado=0.1,
    ativacao=Sigmoid()  # ← Passamos a ativação!
)
print(neuronio_sigmoid)

for epoca in range(10):
    saida, erro = neuronio_sigmoid.treinar(data, esperado)
    print(f"Época {epoca} | saída={saida:.4f} | erro={erro:.4f}")


TESTE 2: Neurônio com função Sigmoid
Neuronio(entradas=10, ativacao=Sigmoid)
Época 0 | saída=0.5073 | erro=0.4927
Época 1 | saída=0.5319 | erro=0.4681
Época 2 | saída=0.5551 | erro=0.4449
Época 3 | saída=0.5770 | erro=0.4230
Época 4 | saída=0.5975 | erro=0.4025
Época 5 | saída=0.6167 | erro=0.3833
Época 6 | saída=0.6346 | erro=0.3654
Época 7 | saída=0.6514 | erro=0.3486
Época 8 | saída=0.6671 | erro=0.3329
Época 9 | saída=0.6817 | erro=0.3183


In [49]:
# ===== TESTE 3: Comparando múltiplas ativações =====
print("\n" + "=" * 50)
print("TESTE 3: Comparando diferentes ativações")
print("=" * 50)

ativacoes_teste = ['degrau', 'sigmoid', 'relu', 'tanh', 'linear']
                  #'degrau', 'sigmoid', 'relU', 'tanh'

for nome_ativacao in ativacoes_teste:
    print(f"\n--- Testando {nome_ativacao.upper()} ---")
    
    ativacao = criar_ativacao(nome_ativacao)
    neuronio = Neuronio(num_entradas=10, taxa_aprendizado=0.1, ativacao=ativacao)
    
    # Treina por 5 épocas
    for epoca in range(5):
        saida, erro = neuronio.treinar(data, esperado)
    
    # Mostra resultado final
    _, saida_final = neuronio.forward(data)
    print(f"Saída final: {saida_final}")


TESTE 3: Comparando diferentes ativações

--- Testando DEGRAU ---
Saída final: 1

--- Testando SIGMOID ---
Saída final: 0.6214499425263434

--- Testando RELU ---
Saída final: 0.9383714350865727

--- Testando TANH ---
Saída final: 0.640907040360849

--- Testando LINEAR ---
Saída final: 0.9536696923479657


In [ ]:
# PyTorch faz exatamente isso!
import torch.nn as nn

# Ativações separadas
ativacao = nn.ReLU()  # ou nn.Sigmoid(), nn.Tanh()...

# Passadas para camadas
camada = nn.Linear(10, 1)
saida = ativacao(camada(entrada))  # ← Mesma ideia!


Saída final: 0.9986959177732646
Erro: 0.9983698972165809
Saída final: 0.9989567342186116
Erro: 0.9986959177732646
Saída final: 0.9991653873748894
Erro: 0.9989567342186116
Saída final: 0.9993323098999115
Erro: 0.9991653873748894
Saída final: 0.9994658479199292
Erro: 0.9993323098999115
